In [1]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial import distance

In [2]:
import pathlib
pathlib.Path().resolve()

WindowsPath('C:/Users/anoda/Documents/Wix')

In [3]:
#df = pd.read_csv('Cerebelly Local Test Control Market Selection Input Data 20231027.csv')
df = pd.read_excel('C:/Users/anoda/Documents/Wix/Wix - Control Market Input.xlsx', sheet_name='Sheet1')

In [10]:
df.columns

Index(['Market', 'BDI', 'Target Population (Adults 25-54)',
       'SMB Per T Pop Index.', 'Selected Market \n(Y/blank)'],
      dtype='object')

In [11]:
columns_check = df.columns.difference(['Selected Market \n(Y/blank)'])

df1 = df.dropna(subset=columns_check)

In [12]:
columns_check

Index(['BDI', 'Market', 'SMB Per T Pop Index.',
       'Target Population (Adults 25-54)'],
      dtype='object')

In [13]:
numeric_cols = df1.select_dtypes(include=['number'])
df1[numeric_cols.columns] = (numeric_cols - numeric_cols.mean()) / numeric_cols.std()


In [14]:
df1.columns

Index(['Market', 'BDI', 'Target Population (Adults 25-54)',
       'SMB Per T Pop Index.', 'Selected Market \n(Y/blank)'],
      dtype='object')

In [15]:
cos_sim = cosine_similarity(df1.iloc[:,1:3])

In [16]:
test_df = df1[df1['Selected Market \n(Y/blank)']=="Y"]

In [17]:
cos_sim.shape

(176, 176)

In [18]:
cos_sim = pd.DataFrame(cos_sim).set_index(df1['Market'])

In [19]:
cos_sim.columns = df1['Market']

In [20]:
cos_out = (cos_sim + 1) * 0.5

In [21]:
cos_out.to_csv('cos_sim_results.csv')

In [22]:
def top_5_similar(df):
    top_5 = {}
    for index, row in df.iterrows():
        top_similar = row.sort_values(ascending=False).drop(index).head(5)
        top_5[index] = top_similar.index.tolist()
    return top_5

top_5_similar_dmas = top_5_similar(cos_out)

In [27]:
df_top_5 = pd.DataFrame.from_dict(top_5_similar_dmas, orient='index').reset_index()
df_top_5.columns = ['DMA', 'Most Similar 1', 'Most Similar 2', 'Most Similar 3', 'Most Similar 4', 'Most Similar 5']

# Export the DataFrame to CSV
df_top_5.to_csv('top_5_similar_dmas.csv', index=False)

In [24]:
distances_df = pd.DataFrame()
for idx1, row1 in test_df[numeric_cols.columns].iterrows():
    row_distances = []
    for idx2, row2 in df1[numeric_cols.columns].iterrows():
        euclidean_dist = distance.euclidean(row1, row2)
        row_distances.append(euclidean_dist)
    distances_df[f'Row {idx1 + 1}'] = row_distances

In [25]:
distances_df.set_index(df1['Market'], inplace=True)
distances_df.columns = test_df['Market']

In [26]:
top_5_similar_dmas

{'Seattle-Tacoma WA': ['San Antonio TX',
  'San Francisco-Oakland-San Jose CA',
  'Phoenix AZ',
  'Houston TX',
  'Dallas-Ft. Worth TX'],
 'Boise ID': ['Baton Rouge LA',
  'Wilmington NC',
  'Jackson TN',
  'Amarillo TX',
  'Columbus GA'],
 'Nashville TN': ['Minneapolis-St. Paul MN',
  'Milwaukee WI',
  'Columbus OH',
  'Portland OR',
  'Boston MA-Manchester NH'],
 'Oklahoma City OK': ['New Orleans LA',
  'Grand Rapids-Kalamazoo-Battle Creek MI',
  'West Palm Beach-Ft. Pierce FL',
  'Harrisburg-Lancaster-Lebanon-York PA',
  'Greensboro-High Point-Winston Salem NC'],
 'Louisville KY': ['Fresno-Visalia CA',
  'Albuquerque-Santa Fe NM',
  'Memphis TN',
  'Greensboro-High Point-Winston Salem NC',
  'Harrisburg-Lancaster-Lebanon-York PA'],
 'Monterey-Salinas CA': ['Providence-New Bedford,MA',
  'Austin TX',
  'Zanesville OH',
  'Las Vegas NV',
  'Yakima-Pasco-Richland-Kennewick WA'],
 'Zanesville OH': ['Yakima-Pasco-Richland-Kennewick WA',
  'Bend OR',
  'Lafayette IN',
  'Gainesville FL',
